# Introduction to Flux Balance Analysis

Author: Daniel Machado, NTNU

License: [CC BY-SA 4.0](http://creativecommons.org/licenses/by-sa/4.0/)

-------

In this tutorial:

- You will learn how to perform flux balance analysis 
using a model of the [central carbon metabolism of *E. coli*](https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1). This is a very simple model, mostly used for educational purposes.

- You will use the [ReFramed](https://github.com/cdanielmachado/reframed) python library for metabolic modeling. You can check the [online documentation](https://reframed.readthedocs.io) for more details.

- You will execute pre-defined pieces of code, but you are strongly encouraged to modify the code and explore on your own.

## Step 1: Loading a model

We begin by loading the model, which is stored in SBML format:

In [ ]:
from reframed import load_cbmodel
model = load_cbmodel('../files/e_coli_core.xml')

Let's learn some details about the model:

In [ ]:
model.summary()

## Step 2: Running an FBA simulation

The most simple thing you can do with a constraint-based model is to run a flux balance analysis (FBA) simulation.

> **Note:** unless you explicitely make some changes, the simulation will use the default objective function (biomass maximization) and environmental conditions (aerobic growth in M9 minimal medium with glucose) that came pre-defined in this model.

In [ ]:
from reframed import FBA
solution = FBA(model)

The solution object contains the objective value (the growth rate in this case), and the respective flux distribution. 

In [ ]:
print(solution)

You can inspect all the fluxes, or select some you are interested in. Let's look at the uptake and secretion rates (which are combined in the form of *exchange* reactions).

In [ ]:
solution.show_values(pattern="R_EX", sort=True)

We can see that *E. coli* is consuming glucose and oxygen and producing CO2.

## Step 3: Visualizing fluxes with Escher

Escher is a really nice tool for displaying fluxes in a metabolic map. It makes your life easier, especially when comparing flux distributions after performing some perturbations (such as gene deletions or changes in the growth medium).

In [ ]:
from reframed import fluxes2escher
fluxes2escher(solution.values)

## Step 4: Changing the environmental conditions

You can change growth conditions either by modifying the flux bounds of the reactions directly in the model or by supplying those constraints as an argument to the FBA simulation method.

Let's observe what happens if we remove oxygen uptake to simulate anaerobic growth:

In [ ]:
solution2 = FBA(model, constraints={'R_EX_o2_e':0})

In [ ]:
print(solution2)

In [ ]:
solution2.show_values(pattern="R_EX", sort=True)

As expected, *E. coli* switched to a fermentation mode, which resulted in the secretion of fermentation products and a decrease in growth rate.

Again, we can see it better by displaying the flux distribution in a metabolic map.

In [ ]:
fluxes2escher(solution2.values)

## Step 5: Comparing simulations with experimental data

Ok, so hopefully you are now convinced that metabolic models and FBA simulations are easy and fun to play with. But can we really trust these flux predictions? 

There is only one way to know. Let's compare them with experimentally measured fluxes. 

[Gerosa and co-workers](https://www.sciencedirect.com/science/article/pii/S2405471215001465) have measured fluxes in *E. coli* growing with different carbon sources. 

![Flux data](../files/gerosa2015.png)



Let's load the fluxomics data that has been stored as a CSV file...

In [ ]:
import pandas as pd
fluxomics = pd.read_csv('../files/gerosa2015.csv', index_col=0)

fluxomics.sample(5) # print 5 random entries

We will constrain the model using only the respective uptake rate for each substrate and see how well it predicts the growth rate and all the other fluxes.

Unfortunately, our model only contains 5 of the 8 substrates used in the paper.

In [ ]:
uptake_reactions = {
    'Acetate': 'R_EX_ac_e',
    'Fructose': 'R_EX_fru_e',
    'Glucose': 'R_EX_glc__D_e',
    'Pyruvate': 'R_EX_pyr_e',
    'Succinate': 'R_EX_succ_e',
}

growth_rates = {
    'Acetate': 0.29,
    'Fructose': 0.49,
    'Glucose': 0.65,
    'Pyruvate': 0.39,
    'Succinate': 0.51,
}

We need to remove glucose from the pre-defined medium, by setting the lower bound of the exchange reaction to zero: 

In [ ]:
model.reactions.R_EX_glc__D_e.lb = 0

Now let's run simulations for all the five conditions. 

> The code below might look a bit complicated. Don't worry about that for now.

In [ ]:
simulated = {}
print('Condition \tGrowth \tPredicted')

for condition, rxn_id in uptake_reactions.items():
    uptake_rate = fluxomics.loc[rxn_id, condition]
    solution = FBA(model, constraints={rxn_id: uptake_rate})
    simulated[f'{condition}_sim'] = solution.values
    print(f'{condition} \t{growth_rates[condition]} \t{solution.fobj:.2f}')
    
combined = pd.concat([fluxomics, pd.DataFrame(simulated)], axis=1, join='inner')

It seems that, in general, the model predicted higher growth rates than what was measured. 

> Why do you think this has happened?

One of the limitations of FBA is that it does not predict overflow metabolism, unless we explicitly add additional constraints.

So let's now additionally constrain the acetate secretion rate as well and see if our predictions improve.

In [ ]:
simulated = {}
print('Condition \tGrowth \tPredicted')

for condition, rxn_id in uptake_reactions.items():
    constraints = {
        rxn_id: fluxomics.loc[rxn_id, condition],
        'R_EX_ac_e': fluxomics.loc['R_EX_ac_e', condition],
    }
    solution = FBA(model, constraints=constraints)
    simulated[f'{condition}_sim'] = solution.values
    print(f'{condition} \t{growth_rates[condition]} \t{solution.fobj:.2f}')
    
combined2 = pd.concat([fluxomics, pd.DataFrame(simulated)], axis=1, join='inner')

There is some improvement in the prediction of growth rates. But what about the fluxes? 
How well are they predicted, and does this also improve when we constrain acetate secretion?

The code below plots measured vs predicted fluxes, before and after constraining acetate secretion.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 5, figsize=(15, 6))

for i, condition in enumerate(uptake_reactions):
    combined.plot.scatter(condition, f'{condition}_sim', ax=axs[0,i])
    axs[0,i].plot([-20, 20], [-20, 20], 'k--', alpha=0.3)
    
    combined2.plot.scatter(condition, f'{condition}_sim', ax=axs[1,i])
    axs[1,i].plot([-20, 20], [-20, 20], 'k--', alpha=0.3)

fig.tight_layout()

You have reached the end of this tutorial. Feel free to go back and try different things.